In [1]:
# importing modules
import numpy as np
import numpy.linalg as LA
from sklearn import svm
from scipy.spatial.distance import pdist, squareform
import scipy

In [15]:
f = open("/home/tandon/IIIT-H/3rd/SMAI/arcene/arcene_train.data")
# f = open("/home/tandon/IIIT-H/3rd/SMAI/dorothea/dorothea_train.data")
X = np.zeros((100, 10000))
row = 0
col = 0
for line in f:
    for token in line.split():
        X[row, col] = int(token)
        col += 1
    col = 0
    row += 1
f.close()

In [3]:
# read the class labels associated with the training data
# f = open("/home/tandon/IIIT-H/3rd/SMAI/arcene/arcene_train.labels")
labels = np.zeros(100)
i = 0
for line in f:
    digit = float(line.strip())
    labels[i] = digit
    i += 1
print(labels.shape)
f.close()

(100,)


In [4]:
labels[4]

-1.0

In [5]:
# Creating the RBF Kernel matrix
def kernel_pca(X, gamma, components):
    n = X.shape[0]
    K = np.zeros((n, n))
#     for i in range(n):
#         for j in range(n):
#             difference = X[i,:] - X[j,:]
#             dif_norm = LA.norm(difference)**2
# #             print('dif norm', dif_norm)
#             prod = -0.5 * dif_norm / variance
#             K[i, j] = np.exp(prod)
# #             print('k val', K[i, j])
    sq_dists = pdist(X, 'sqeuclidean')
    mat_sq_dists = squareform(sq_dists)
    K = scipy.exp(-gamma * mat_sq_dists)
    onen = np.ones((n, n))/n
    # Centering the Kernel matrix
    K = K - onen.dot(K) - K.dot(onen) + onen.dot(K).dot(onen)
    eigen_values, eigen_vectors = LA.eig(K)
#     print('Eigen vecs', eigen_values.shape)
    # Contains indices for largest to smallest eigen values
    idx = eigen_values.argsort()[::-1]
    alphas = eigen_vectors[:, idx[0:components]]
    lambdas = eigen_values[idx[0:components]]
    return alphas,lambdas

def normalize(alphas, lambdas):
    return alphas/lambdas
        

In [6]:
def project_point(point, X, alphas, gamma):
    kvec = np.array([np.sum((point - row)**2) for row in X])
    kvec = np.exp(- gamma * kvec)
    return alphas.T.dot(kvec)
    print('Projected point')

In [7]:
# finding the normalized eigen vectors associated with the training data
components = 10
gamma = 1/(np.var(X)*50)
alphas, lambdas = kernel_pca(X, gamma, components)
normalized_alphas = normalize(alphas, lambdas)
normalized_alphas

array([[  1.58432056e-02+0.j,   1.91407428e-02+0.j,   6.25438582e-03+0.j,
         -2.57368466e-02+0.j,  -3.95538860e-02+0.j,   8.17220825e-02+0.j,
         -6.22439510e-02+0.j,   6.11771875e-01+0.j,  -1.38337471e-01+0.j,
         -1.73789863e-02+0.j],
       [ -1.73789009e-02+0.j,   2.08033419e-02+0.j,   3.67530392e-03+0.j,
         -1.44155910e-02+0.j,  -1.78675920e-02+0.j,  -2.83952626e-02+0.j,
         -1.70314844e-02+0.j,  -1.11736091e-02+0.j,   2.82127857e-02+0.j,
          1.34384914e-02+0.j],
       [  4.03538405e-01+0.j,  -3.48564062e-02+0.j,  -4.51999162e-04+0.j,
          1.60697609e-03+0.j,   1.40849316e-03+0.j,  -7.85164125e-04+0.j,
         -3.44016818e-03+0.j,   2.19399831e-02+0.j,  -2.56569737e-03+0.j,
          9.24421141e-04+0.j],
       [ -1.72626162e-02+0.j,   2.06092111e-02+0.j,   3.57786956e-03+0.j,
         -1.40220028e-02+0.j,  -1.73231301e-02+0.j,  -2.72604897e-02+0.j,
         -1.57380818e-02+0.j,  -1.01468713e-02+0.j,   2.53035595e-02+0.j,
          1.1521049

In [9]:
# training the SVM classifier
X_projected = np.zeros((100, components))
clf = svm.SVC()
for i in range(100):
    X_projected[i,:] = project_point(X[i], X, normalized_alphas, gamma)
clf.fit(X_projected, labels)

/home/tandon/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: ComplexWarning: Casting complex values to real discards the imaginary part


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
# read the validation data
f = open("/home/tandon/IIIT-H/3rd/SMAI/arcene/arcene_valid.data")
X_valid = np.zeros((100, 10000))
row = 0
col = 0
for line in f:
    for token in line.split():
        X_valid[row, col] = int(token)
        col += 1
    col = 0
    row += 1
f.close()

In [11]:
# read the class labels associated with the validation data
f = open("/home/tandon/IIIT-H/3rd/SMAI/arcene/arcene_valid.labels")
labels_valid = np.zeros(100)
i = 0
for line in f:
    digit = float(line.strip())
    labels_valid[i] = digit
    i += 1
f.close()

In [13]:
# validating our training 
correct = 0
for i in range(100):
    projection = project_point(X_valid[i], X_valid, normalized_alphas, gamma)
#     print('Projection', projection)
    if clf.predict(projection)[0] > 0:
        if labels_valid[i] > 0:
            correct += 1
    else:
        if labels_valid[i] < 0:
            correct += 1
print('Correct Efficiency: ', correct/100)
            
    
    

/home/tandon/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:373: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
/home/tandon/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/tandon/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/tandon/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: Deprecatio

Correct Efficiency:  0.56


/home/tandon/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/tandon/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/tandon/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single samp

### For Kernel PCA

#### For Ircene dataset

* Accuracy comes as 56% for k = 100
* Accuracy comes as 56% for k = 10

#### For Dorothea dataset

* Accuracy comes as 90.25% for k = 100
* Accuracy comes as 90.25% for k = 10

### Observation

One thing observed was that, all the data was getting classified as either +1 or -1. May be RBF kernel is not able to project the data points linearly